<a id='top'></a>
# Imaging Mode Data Calibration: Part 1 - Ramps to Slopes
---
**Author**: Bryan Hilbert (hilbert@stsci.edu)| **Latest Update**: 20 May 2021

<div class="alert alert-block alert-warning">
    <h3><u><b>Notebook Goals</b></u></h3>
    <ul>Working with the Stage 1 Calibration Pipeline, we will:</ul>    
<ul>
    <li>Look at the different ways to call the pipeline</li>
    <li>Examine exactly what each pipeline step does to the science data</li>    
</ul>
</div>

## Table of Contents
* [Introduction](#intro)
* [Pipeline Resources and Documentation](#resources)
   * [Installation](#installation)
   * [Reference Files](#reference_files)
* [Imports](#imports)
* [Convenience Functions](#convenience_functions)
* [Download Data](#download_data)
* [Methods for calling steps/pipelines](#calling_methods)
* [Parameter reference files](#parameter_reffiles)
* [calwebb_detector1 - Ramps to slopes](#detector1) 
   * [Run the entire pipeline](#detector1_at_once)
       * [run() method](#run_method)
       * [call() method](#call_method)
       * [command line](#command_line)
       * [Exercise 1](#exercise1)
   * [Run the individual pipeline steps](#detector1_step_by_step)
       * [The `Data Quality Initialization` step](#dq_init)
       * [The `Saturation Flagging` step](#saturation)
       * [The `Superbias Subtraction` step](#superbias)
       * [The `Reference Pixel Subtraction` step](#refpix)
           * [Exercise 2](#exercise2)
       * [The `Linearity Correction` step](#linearity)
       * [The `Persistence Correction` step](#persistence)
       * [The `Dark Current Subtraction` step](#dc)
       * [The `Cosmic Ray Flagging` step](#jump)
       * [The `Ramp_Fitting` step](#ramp_fitting)
* [Bonus Topic: Logging](#logging)
* [Exercise solutions](#exercise_solutions)
    * [Exercise 1 solution](#exercise1_solution)
    * [Exercise 2 solution](#exercise2_solution)

<a id='intro'></a>
## Introduction

This notebook covers part 1 of the imaging mode data calibration module. In this notebook we'll review Stage 1 of the JWST calibration pipeline, also known as *calwebb_detector1*. 

The [Stage 1 pipeline](https://jwst-pipeline.readthedocs.io/en/stable/jwst/pipeline/calwebb_detector1.html#calwebb-detector1) applies basic detector-level corrections to all exposure types (imaging, spectroscopic, coronagraphic, etc.). It is applied to one exposure at a time, beginning with an uncalibrated multiaccum ramp (*_uncal.fits file*). It is sometimes referred to as “ramps-to-slopes” processing. 

Each input raw data file is composed of one or more ramps (integrations) containing increasing count values from the non-destructive detector readouts. For details on multiaccum files and data collection, see the JDox page on [how up-the-ramp readouts work](https://jwst-docs.stsci.edu/understanding-exposure-times#UnderstandingExposureTimes-uptherampHowup-the-rampreadoutswork).

The output is a corrected but uncalibrated countrate or slope image (*_rate.fits and _rateints.fits file*). In this case, "calibrated/uncalibrated" is based on the units of the data. Any data in units of DN or DN/sec are considered uncalibrated. After the flux calibration step of the Stage 2 pipeline is run and the data units become physical units (e.g. MJy/str), then the data are said to be calibrated.

To illustrate how the steps of the pipeline change the input data, we will download a sample file and run it through the pipeline, examining the results at several places along the way.

All JWST data, regardless of instrument and observing mode, are processed through the Stage 1 pipeline. The corrections performed are the same across all near-IR instruments. There are several additional MIRI-specific steps. For the purposes of this notebook, our example file will be NIRCam data. We will also provide an example MIRI file that can be used in a separate exercise.

<a id='resources'></a>
## Pipeline Resources and Documentation

There are several different places to find information on installing and running the pipeline. This notebook will summarize each Stage 1 pipeline step. To find more in-depth instructions use the links below.

* [JWST Documentation (JDox) for the Stage 1 pipeline](https://jwst-docs.stsci.edu/jwst-data-reduction-pipeline/stages-of-processing/calwebb_detector1) including short a short summary of what each step does.

* [High-level description of all pipeline stages and steps](https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/main.html) from the pipeline software documentation website.

* [`jwst` package documentation](https://jwst-pipeline.readthedocs.io/en/latest/jwst/introduction.html) including how to run the pipeline, input/output files, etc.

* [`jwst` package GitHub repository, with installation instructions](https://github.com/spacetelescope/jwst/blob/master/README.md)

* [**Help Desk**](https://stsci.service-now.com/jwst?id=sc_cat_item&sys_id=27a8af2fdbf2220033b55dd5ce9619cd&sysparm_category=e15706fc0a0a0aa7007fc21e1ab70c2f): **If you have any questions or problems regarding the pipeline, submit a ticket to the Help Desk**

<a id='installation'></a>
### Installation

<div class="alert alert-block alert-info">
    During the JWebbinar, we will be working in a pre-existing environment where the <b>jwst</b> package has already been installed, so you won't need to install it yourself.
</div>

<div class="alert alert-block alert-warning">
If you wish to run this notebook outside of this JWebbinar, you will have to first install the <b>jwst</b> package.<br>

For more detailed instructions on the various ways to install the package, see the [installation instructions](https://github.com/spacetelescope/jwst/blob/master/README.md) on GitHub.

The easiest way to install the pipeline is via `pip`. Below we show how to create a new conda environment, activate that environment, and then install the latest released version of the pipeline. You can name your environment anything you like. In the lines below, replace `<env_name>` with your chosen environment name.

>`conda create -n <env_name> python`<br>
>`conda activate <env_name>`<br>
>`pip install jwst`

If you wish to install the development version of the pipeline, which is more recent than (but not as well tested compared to) the latest released version:

>`conda create -n <env_name> python`<br>
>`conda activate <env_name>`<br>
>`pip install git+https://github.com/spacetelescope/jwst`
    
</div>

<a id='reference_files'></a>
### Reference Files

[Calibration reference files](https://jwst-docs.stsci.edu/data-processing-and-calibration-files/calibration-reference-files) are a collection of FITS and ASDF files that are used to remove instrumental signatures and calibrate JWST data. For example, the dark current reference file contains a multiaccum ramp of dark current signal to be subtracted from the data during the dark current subtraction step. 

When running a pipeline or pipeline step, the pipeline will automatically look for any required reference files in a pre-defined local directory. If the required reference files are not present, they will automatically be downloaded from the Calibration Reference Data System (CRDS) at STScI.

<div class="alert alert-block alert-info">
    During the JWebbinar, our pre-existing existing environment is set up to correctly use and store calibration reference files, and you do not need to set the environment variables below.
</div>
    
<div class="alert alert-block alert-warning">
If you wish to run this notebook outside of this JWebbinar, you will have to specify a local directory in which to store reference files, along with the server to use to download the reference files from CRDS. To accomplish this, there are two environment variables that should be set prior to calling the pipeline. These are the CRDS_PATH and CRDS_SERVER_URL variables. In the example below, reference files will be downloaded to the "crds_cache" directory under the home directory.

>`$ export CRDS_PATH=$HOME/crds_cache`<br>
>`$ export CRDS_SERVER_URL=https://jwst-crds.stsci.edu`<br>
OR:<br>
`os.environ["CRDS_PATH"] = "/user/myself/crds_cache"`<br>
`os.environ["CRDS_SERVER_URL"] = "https://jwst-crds.stsci.edu"`<br>

The first time you run the pipeline, the CRDS server should download all of the context and reference files that are needed for that pipeline run, and dump them into the CRDS_PATH directory. Subsequent executions of the pipeline will first look to see if it has what it needs in CRDS_PATH and anything it doesn't have will be downloaded from the STScI cache. 
</div>

<div class="alert alert-block alert-warning">NOTE: Users at STScI should automatically have access to the Calibration Reference Data System (CRDS) cache for running the pipeline, and can skip setting these environment variables.</div>

[Top of Notebook](#top)

<a id='imports'></a>
## Imports

Import packages necessary for this notebook

In [ ]:
# Packages that allow us to get information about objects:
import asdf
import copy
import os
import shutil

# Numpy library:
import numpy as np

# For downloading data
import requests

# Astropy tools:
from astropy.io import fits
from astropy.utils.data import download_file
from astropy.visualization import ImageNormalize, ManualInterval, LogStretch

Set up matplotlib for plotting

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl

# Use this version for non-interactive plots (easier scrolling of the notebook)
%matplotlib inline

# Use this version (outside of Jupyter Lab) if you want interactive plots
#%matplotlib notebook

# These gymnastics are needed to make the sizes of the figures
# be the same in both the inline and notebook versions
%config InlineBackend.print_figure_kwargs = {'bbox_inches': None}

mpl.rcParams['savefig.dpi'] = 80
mpl.rcParams['figure.dpi'] = 80

Import JWST pipeline-related modules

In [ ]:
# List of possible data quality flags
from jwst.datamodels import dqflags

# The entire calwebb_detector1 pipeline
from jwst.pipeline import calwebb_detector1

# Individual steps that make up calwebb_detector1
from jwst.dq_init import DQInitStep
from jwst.saturation import SaturationStep
from jwst.superbias import SuperBiasStep
from jwst.ipc import IPCStep                                                                                    
from jwst.refpix import RefPixStep                                                                
from jwst.linearity import LinearityStep
from jwst.persistence import PersistenceStep
from jwst.dark_current import DarkCurrentStep
from jwst.jump import JumpStep
from jwst.ramp_fitting import RampFitStep
from jwst import datamodels

Check which version of the pipeline we are running:

In [ ]:
import jwst
print(jwst.__version__)

<a id='convenience_functions'></a>
## Define convenience functions and parameters

Here we define some functions and some parameters that we will use repeatedly throughout the notebook.

In [ ]:
# To make everything easier, all files saved by the pipeline
# and pipeline steps will be saved to the working directory.
# This will be more important for the level 2 and 3 pipelines
# once we are working with association files.
output_dir = './'

In [ ]:
# Make sure the output directory exists before downloading any data
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [ ]:
def download_files(files, output_directory, force=False):
    """Given a tuple or list of tuples containing (URL, filename),
    download the given files into the current working directory.
    Downloading is done via astropy's download_file. A symbolic link
    is created in the specified output dirctory that points to the
    downloaded file.
    
    Parameters
    ----------
    files : tuple or list of tuples
        Each 2-tuple should contain (URL, filename), where
        URL is the URL from which to download the file, and
        filename will be the name of the symlink pointing to
        the downloaded file.
        
    output_directory : str
        Name of the directory in which to create the symbolic
        links to the downloaded files
        
    force : bool
        If True, the file will be downloaded regarless of whether
        it is already present or not.
                
    Returns
    -------
    filenames : list
        List of filenames corresponding to the symbolic links
        of the downloaded files
    """
    # In the case of a single input tuple, make it a
    # 1 element list, for consistency.
    filenames = []
    if isinstance(files, tuple):
        files = [files]
        
    for file in files:
        filenames.append(file[1])
        if force:
            print('Downloading {}...'.format(file[1]))
            demo_file = download_file(file[0], cache='update')
            # Make a symbolic link using a local name for convenience
            if not os.path.islink(os.path.join(output_directory, file[1])):
                os.symlink(demo_file, os.path.join(output_directory, file[1]))
        else:
            if not os.path.isfile(os.path.join(output_directory, file[1])):
                print('Downloading {}...'.format(file[1]))
                demo_file = download_file(file[0], cache=True)
                # Make a symbolic link using a local name for convenience
                os.symlink(demo_file, os.path.join(output_directory, file[1]))
            else:
                print('{} already exists, skipping download...'.format(file[1]))
                continue
    return filenames    

In [ ]:
def plot_jump(signal, jump_group, xpixel=None, ypixel=None, slope=None):
    """Function to plot the signal up the ramp for a
    pixel and show the location of flagged jumps.
    
    Parameters
    ----------
    signal : numpy.ndarray
        1D array of signal values
        
    jump_group : list
        List of boolean values whether a jump is present or
        not in each group
        
    slope : numpy.ndarray
        1D array of signal values constructed from the slope
    """
    groups = np.arange(len(signal))
    fig = plt.figure(figsize=(6, 6))
    ax = plt.subplot()

    plt.plot(groups, signal, marker='o', color='black')
    plt.plot(groups[jump_group], signal[jump_group], marker='o', color='red',
             label='Flagged Jump')
    
    if slope is not None:
        plt.plot(groups, slope, marker='o', color='blue', label='Data from slope')
        
    plt.legend(loc=2)

    plt.xlabel('Groups')
    plt.ylabel('Signal (DN)')
    fig.tight_layout()
    plt.subplots_adjust(top=0.95)
    
    if xpixel and ypixel:
        plt.title('Pixel ('+str(xpixel)+','+str(ypixel)+')')

In [ ]:
def plot_jumps(signals, jump_groups, pixel_loc, slopes=None):
    """Function to plot the ramp and show the jump location
    for several pixels. For simplicity, let's force the input
    number of pixels to be a square. 
    
    Parameters
    ----------
    signals : numpy.ndarray
        2D array (groups x pix) of signal values
        
    jump_groups : numpy.ndarray
        2D array containing boolean entries for each group of
        each pixel, describing where the jumps were found
        
    pixel_loc : list
        List of 2-tuples containing the (x, y)
        location of the pixels with the jumps
        
    slopes : numpy.ndarray
        2D array (groups x pix) of linear signal values
        If not None, these will be overplotted onto the
        plots of signals
    """
    num_group, num_pix = signals.shape
    root = np.sqrt(num_pix)
    if int(root + 0.5) ** 2 != num_pix:
        raise ValueError('Number of pixels input should be a square.')
    
    root = int(root)
    groups = np.arange(num_group)
    fig, axs = plt.subplots(root, root, figsize=(10, 10))

    for index in range(len(pixel_loc)):
        i = int(index % root)
        j = int(index / root)
        axs[i, j].plot(groups, signals[:, index], marker='o', color='black')
        j_grp = jump_groups[:, index]
        axs[i, j].plot(groups[j_grp], signals[j_grp, index],
                       marker='o', color='red')
        
        if slopes is not None:
            axs[i, j].plot(groups, slopes[:, index], marker='o', color='blue')
        
        axs[i, j].set_title('Pixel ({}, {})'.format(pixel_loc[index][1], pixel_loc[index][0]))
        
    plt.xlabel('Groups')
    plt.ylabel('Signal (DN)')
    fig.tight_layout()

In [ ]:
def plot_ramp(groups, signal, xpixel=None, ypixel=None, title=None):
    """Function to plot the up the ramp signal for a pixel.
    
    Parameters
    ----------
    groups : numpy.ndarray
        1D array of group numbers. X-axis values.
        
    signal : numpy.ndarray
        1D array of pixel signal values.
        
    xpixel : int
        X-coordinate of the pixel being plotted. Used for legend only.
        
    ypixel : int
        Y-coordinate of the pixel being plotted. Used for legend only.
        
    title : str
        String to use for the plot title
    """    
    fig = plt.figure(figsize=(8, 8))
    ax = plt.subplot()
    if xpixel and ypixel:
            plt.plot(groups, signal, marker='o',
                     label='Pixel ('+str(xpixel)+','+str(ypixel)+')') 
            plt.legend(loc=2)

    else:
        plt.plot(groups, signal, marker='o')
        
    plt.xlabel('Groups')
    plt.ylabel('Signal (DN)')
    fig.tight_layout()
    plt.subplots_adjust(left=0.15)
    
    if title:
        plt.title(title)

In [ ]:
def plot_ramps(groups, signal1, signal2, label1=None, label2=None, title=None):
    """Function to plot the up the ramp signal for two pixels
    on a single plot.
    
    Parameters
    ----------
    groups : numpy.ndarray
        1D array of group numbers. X-axis values.
        
    signal1 : numpy.ndarray
        1D array of signal values for first pixel
        
    signal2 : numpy.ndarray
        1D array of signal values for second pixel
        
    label1 : str
        Label to place in the legend for pixel1
        
    label2 : str
        Label to place in the legend for pixel2
        
    title : str
        String to place in the title of the plot
    """    
    fig = plt.figure(figsize=(6, 6))
    ax = plt.subplot()
    if label1:
        plt.plot(groups, signal1, marker='o', color='black', label=label1)
    else:
        plt.plot(groups, signal1, marker='o', color='black')
    if label2:
        plt.plot(groups, signal2, marker='o', color='red', label=label2)
    else:
        plt.plot(groups, signal2, marker='o', color='red')
    if label1 or label2:
        plt.legend(loc=2)
        
    plt.xlabel('Groups')
    plt.ylabel('Signal (DN)')
    fig.tight_layout()
    plt.subplots_adjust(left=0.15)
    plt.subplots_adjust(top=0.95)
    
    if title:
        plt.title(title)

In [ ]:
def show_image(data_2d, vmin, vmax, xpixel=None, ypixel=None, title=None):
    """Function to generate a 2D, log-scaled image of the data, 
    with an option to highlight a specific pixel (with a red dot).
    
    Parameters
    ----------
    data_2d : numpy.ndarray
        Image to be displayed
        
    vmin : float
        Minimum signal value to use for scaling
        
    vmax : float
        Maximum signal value to use for scaling
        
    xpixel : int
        X-coordinate of pixel to highlight
        
    ypixel : int
        Y-coordinate of pixel to highlight
        
    title : str
        String to use for the plot title
    """
    norm = ImageNormalize(data_2d, interval=ManualInterval(vmin=vmin, vmax=vmax),
                          stretch=LogStretch())
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(1, 1, 1)
    im = ax.imshow(data_2d, origin='lower', norm=norm)
    
    if xpixel and ypixel:
        plt.plot(xpixel, ypixel, marker='o', color='red', label='Selected Pixel')

    fig.colorbar(im, label='DN')
    plt.xlabel('Pixel column')
    plt.ylabel('Pixel row')
    if title:
        plt.title(title)

In [ ]:
def side_by_side(data1, data2, vmin, vmax, title1=None, title2=None, title=None):
    """Show two images side by side for easy comparison. Optionally highlight
    a given pixel with a red dot.
    
    Parameters
    ----------
    data1 : numpy.ndarray
        First image to be displayed
        
    data2 : numpy.ndarray
        Second image to be displayed
        
    vmin : float
        Minimum signal value to use for scaling
        
    vmax : float
        Maximum signal value to use for scaling
            
    title1 : str
        Title to use for first (left) plot
        
    title2 : str
        Title to use for the second (right) plot

    title : str
        String to use for the plot title
    """
    norm = ImageNormalize(data1, interval=ManualInterval(vmin=vmin, vmax=vmax),
                          stretch=LogStretch())

    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(11, 8))
    im = axes[0].imshow(data1, origin='lower', norm=norm)
    im = axes[1].imshow(data2, origin='lower', norm=norm)
    
    axes[0].set_xlabel('Pixel column')
    axes[0].set_ylabel('Pixel row')
    axes[1].set_xlabel('Pixel column')
    
    if title1:
        axes[0].set_title(title1)
    if title2:
        axes[1].set_title(title2)
        
    fig.subplots_adjust(right=0.8)
    cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
    fig.colorbar(im, cax=cbar_ax, label='DN')
    
    if title:
        fig.suptitle(title)    

[Top of Notebook](#top)

<a id='download_data'></a>
## Download Data

For this module, we will use an uncalibrated NIRCam simulated imaging exposure that is stored in Box. Let's grab it:

In [ ]:
uncal_info = ('https://stsci.box.com/shared/static/j46wpyirlbqo30e7c9719ycnuc1qk2lu.fits',
              'jw98765001001_01101_00003_nrcb5_uncal.fits')
uncal_file = download_files(uncal_info, output_dir, force=True)[0]

In [ ]:
# Also download a "trapsfilled" file, which specifies the state of the charge traps at
# the time of the preceding exposure. This will serve as input to the persistence
# step.
persist_info = ('https://stsci.box.com/shared/static/ehkof12d43h6nnpijs2r4tyuzde3nzc9.fits',
                'jw98765001001_01101_00002_nrcb5_trapsfilled.fits')
persist_file = download_files(persist_info, output_dir, force=True)[0]

In [ ]:
# Download example parameter reference files
param_info = [('https://stsci.box.com/shared/static/nmaipgvlferrz95eyksra4zmlhx2m5bm.asdf',
               'detector1pipeline_modified_paramfile.asdf'),
              ('https://stsci.box.com/shared/static/kww0o8d77h2u03b38gtdr34g8vqygx8w.asdf',
               'refpix_paramfile.asdf'),
              ('https://stsci.box.com/shared/static/c95paaz9gsvrwwo3slx8k9qav242idi1.asdf',
               'refpix_no_side_pix_paramfile.asdf')]
detector1_param_reffile, refpix_param_reffile, refpix_param_reffile_no_side_pix = \
    download_files(param_info, output_dir, force=False)

You can also download the example MIRI file if you wish to try the exercise of running it through the pipeline.

In [ ]:
# Download raw MIRI file for notebook exercises
miri_uncal_info = ('https://stsci.box.com/shared/static/rsav0kkg4dhb4y3oth6c0orcafc83qgm.fits',
                   'miri_F770W_exp1_uncal.fits')
miri_uncal_file = download_files(miri_uncal_info, output_dir, force=True)[0]

In [ ]:
# Download a MIRI parameter refrence file for the pipeline
miri_param_info = ('https://stsci.box.com/shared/static/cbsj8v6ull992n4jdkzlg5485arsmars.asdf',
                   'miri_detector1_param_file.asdf')
miri_det1_param_reffile = download_files(miri_param_info, output_dir, force=False)[0]

<a id='calling_methods'></a>
## Methods for calling steps/pipelines

There are three common methods by which the pipeline or pipeline steps can be called. From within python, the `run()` and `call()` methods of the pipeline or step classes can be used. Alternatively, the `strun` command can be used from the command line. Within this notebook, in the section where we [call the entire pipeline](#detector1_at_once), as well as the section where we [call the Reference Pixel Subtraction](#refpix) step, we show examples of all three methods. For the remainder of the pipeline steps, we will focus on using the `run()` method.

When using the `call()` method or `strun`, optional input parameters can be specified via [parameter reference files](#parameter_reffiles). When using the `run()` method, these parameters are instead specified within python.

<a id='default_calls'></a>

<div class="alert alert-block alert-info">
The simplest case: Calling the pipeline with all default parameter values. In these cases, the pipeline falls back to retrieving default parameter values from the pipeline code itself, or by retrieving the default parameter reference file stored in CRDS.
</div>

<div class="alert alert-block alert-info">
    
Use strun to call the pipeline and use all default parameter values:    
```
    strun jwst.pipeline.Detector1Pipeline jw98765001001_01101_00003_nrcb5_uncal.fits
```
</div>

<a id='parameter_reffiles'></a>
## Parameter Reference Files

When calling a pipeline or pipeline step using the `call()` method or the command line, [parameter reference files](https://jwst-pipeline.readthedocs.io/en/stable/jwst/stpipe/config_asdf.html#config-asdf-files) can be used to specify values for input parameters. These reference files are in [asdf](https://asdf.readthedocs.io/en/stable/) format and appear somewhat similar to json files when examined in a text editor. 

Versions of parameter reference files containing default parameter values for each step and pipeline are available in CRDS. When using the `call()` method, if you do not specify a parameter reference file name in the call, the pipeline or step will retrieve and use the appropriate file from CRDS, which will then run the pipeline or step with the parameter values in that file. If you provide the name of a parameter reference file, then the parameter values in that file will take precedence. For any parameter not specified in your parameter reference file, the pipeline will use the default value.

When using `strun`, the parameter reference file is a required input.

Let's take a look at the contents of a parameter reference file. We'll open it using the asdf package, and use the `tree` attribute to see what's inside:

In [ ]:
det1_reffile = asdf.open(detector1_param_reffile)

In [ ]:
det1_reffile.tree
# or:
# det1_reffile.info(max_rows=None)

The top part of the file contains various metadata entries about the file itself. Below that, you'll see a `'name'` entry, which lists `Detector1Pipeline` as the class to which these parameters apply. The next line contains the `parameters` entry, which lists parameters and values attached to the pipeline itself. Below this is the `steps` entry, which contains a list of dictionaries. Each dictionary refers to one step within the pipeline, and specifies parameters and values that apply to that step. If you look through these entries, you'll see the same parameters and values that we specified manually when using the `run()` method above.

In [ ]:
# Don't forget to close the file
det1_reffile.close()

[Top of Notebook](#top)

---
<a id='detector1'></a>
## The calwebb_detector1 pipeline: Ramps to slopes

In the sections below, we will run the Stage 1 pipeline on a single uncalibrated NIRCam file. We will first call the entire [*calwebb_detector1* pipeline](https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_detector1.html#calwebb-detector1) on the file. The pipeline is a wrapper which will string together all of the appropriate steps in the proper order. 

The final output from this call is an uncalibrated slope image which is ready to go into the Stage 2 pipeline. "Uncalibrated" in this case means that the data are in units of DN/sec. In Stage 2 the flux calibration will be applied, at which point the data will be in physical units (e.g. MJy/str) and referred to as "calibrated".

After that, we will go back to the original uncalibrated ramp and manually run it through each of the steps that comprise the Stage 1 pipeline. For each step we will describe in more detail what is going on, and for several of the more interesting steps, we will examine the output.

See [Figure 1](https://jwst-docs.stsci.edu/jwst-data-reduction-pipeline/stages-of-processing/calwebb_detector1) on the calwebb_detector1 algorithm page for a map of which steps are performed on NIR data and which are used for MIRI data.

In [ ]:
input_file_base = os.path.basename(uncal_file).replace('uncal.fits', '')

<a id='detector1_at_once'></a>
### Run the entire `calwebb_detecor1` pipeline

In this section we show how to run the entire calwebb_detector1 pipeline with a single call. In this case the pipeline code can determine which instrument was used to collect the data and runs the appropriate steps in the proper order.

We set parameter values for some of the individual steps, save some outputs, etc, and then call the pipeline.

We will call the pipeline using the `run()` method and while that is running, we will go over the equivalent `call()` and `strun` commands, examine some of the pipeline log entries that are printed to the screen, and then look at the pipeline output.

<a id='run_method'></a>
#### Call the pipeline using the run() method

When using the `run()` method to execute a pipeline (or step), the pipeline class is first instantiated without the data to be processed. Optional input parameters are specified using attributes of the class instance. Finally, the call to the `run()` method is made and the data are supplied.  See here for [more examples of the run() method](https://jwst-pipeline.readthedocs.io/en/stable/jwst/stpipe/call_via_run.html).

The `run()` method does not take any kind of parameter reference file as input. If you wish to set values for various parameters, you must do that manually. Below, we set several parameters in order to show how it's done. 

How do you know what parameters are available to be set and what their default values are? The `spec` property for individual steps will list them. The property is less useful for the pipelines themselves, as it does not show the parameters for the steps comprising the pipeline.

All steps and pipelines have several common parameters that can be set. 

* `save_results` specifies whether or not to save the output of that step/pipeline to a file. The default is False.
* `output_dir` is the directory into which the output files will be saved.
* `output_file` is the base filename to use for the saved result. Note that each step/pipeline will add a custom suffix onto output_file. 

Let's look at the available parameters for the reference pixel subtraction step, and the cosmic ray flagging step, and manually set some of these in our call to `run()`.

In [ ]:
print(RefPixStep.spec)

In [ ]:
print(JumpStep.spec)

<a id='detector1_using_run'></a>

<div class="alert alert-block alert-info">
Finally, let's run the pipeline. The output can be a little overwhelming. There will be multiple log entries printed to the screen for each step.
</div>

In [ ]:
# Create an instance of the pipeline class
detector1 = calwebb_detector1.Detector1Pipeline()

# Set some parameters that pertain to the
# entire pipeline
detector1.output_dir = output_dir
detector1.save_results = True

# Set some parameters that pertain to some of
# the individual steps
detector1.refpix.use_side_ref_pixels = True
detector1.linearity.save_results = True
detector1.jump.rejection_threshold = 6

# Specify the name of the trapsfilled file, which
# contains the state of the charge traps at the end
# of the preceding exposure
detector1.persistence.input_trapsfilled = persist_file

# Call the run() method
run_output = detector1.run(uncal_file)

This cell will take a few minutes to run. While we're waiting, let's look at the other two methods that can be used to call the pipeline. Then we'll come back here and look at the log meessages output by this cell so we can see what happened.

<a id='call_method'></a>
#### Call the pipeline using the call() method

When using the `call()` method, a single command will instantiate and run the pipeline (or step). The input data and optional parameter reference files are supplied in this single command. In this case, any desired input parameters cannot be set after instantiation, as with the `run()` method. See here for [example usage of call() method](https://jwst-pipeline.readthedocs.io/en/stable/jwst/stpipe/call_via_call.html).

The commands below will call the pipeline using the `call()` method and will supply the parameter reference file. Since we just ran the pipeline with the `run()` method above, we won't actually execute the call to `call()`. But if you wish to try it out, use the pull-down menu above to change the cell to be 'Code', and then execute it. (Or, Click 'Cell' > 'Cell Type' > 'Code')

<div class="alert alert-block alert-info">

<b>Method #1:</b>
Provide the name of the observation file, the pipeline-specific input paramters, and the name of the parameter reference file that specifies step-specific parameters
</div>

<div class="alert alert-block alert-info">

<b>Method #2:</b>
In this case, build a nested dictionary that specifies parameter values for various steps, and provide it in the call to call().
</div>

<a id='command_line'></a>
#### Call the pipeline using the command line

Calling a pipeline or step from the command line is similar to using the `call()` method. The data file to be processed, along with an optional parameter reference file and optional parameter/value pairs can be provided to the `strun` command. See here for [additional examples of command line calls](https://jwst-pipeline.readthedocs.io/en/stable/jwst/introduction.html?highlight=%22command%20line%22#running-from-the-command-line).

The cells below contains two different command line commands that use `strun` to call the calwebb_detector1 pipeline. 

<div class="alert alert-block alert-info">

<b>Method #1:</b>
We provide the name of the pipeline class, the observation file, and explicitly set pipeline- and step-specific parameters. You can see that the command quickly becomes quite large with the added parameter settings. 
    
```
    strun jwst.pipeline.Detector1Pipeline jw98765001001_01101_00003_nrcb5_uncal.fits --save_results=True --output_dir='./' --steps.refpix.use_side_ref_pixels=True --steps.linearity.save_results=True --steps.jump.rejection_threshold=6 
```
</div>

<div class="alert alert-block alert-info">

<b>Method #2:</b>
This version of the command is much more succinct, as the parameter values to be set are all contained within the parameter reference file. The pipeline class is also contained in the parameter reference file, so there is no need to specify it in the command itself.
    
```
    strun detector1pipeline_modified_paramfile.asdf jw98765001001_01101_00003_nrcb5_uncal.fits
```
</div>

### Examine the outputs

The primary output of the calwebb_detector1 pipeline is a file containing a rate image for the exposure. The units of the data are ADU/sec.  

In [ ]:
# Generate the rate file name from the uncal file name
rate_file = uncal_file.replace('uncal.fits', 'rate.fits')

In [ ]:
rate_file

In [ ]:
# Use getdata to quickly read in the science data from the rate file
rate_data = fits.getdata(rate_file)

In [ ]:
# Look at the rate image
show_image(rate_data, 0.5, 10)

Also, since we set the linearity step's `save_results` parameter to True in our calls above, the pipeline saved the output of the linearity step. In this case, the output file will have the same name as the input uncal file, but with the suffix 'linearity' rather than 'uncal'. 

**NOTE:** This differs slightly from the case where we call the linearity step itself and save the results. In that case, as we will see in the [linearity](#linearity) section, the output file will have the suffix 'linearitystep' rather than 'linearity'.

In [ ]:
# Generate the linear step file name from the uncal file name
linear_file = rate_file.replace('rate.fits', 'linearity.fits')

In [ ]:
# Read in the science data from the linear file
lin_data = fits.getdata(linear_file)

We will look in more detail at the effects of the linearity correction step in the [linearity](#linearity) section below. For now, let's just look at the final group of the integration.

In [ ]:
# Let's look at the data in the final group of the linearized data:
show_image(lin_data[0, -1, :, :], 100, 10000)

[Top of Notebook](#top)

<a id='exercise1'></a>
### Exercise 1: Run calwebb_detector1 on MIRI data

Try running the pipeline on the raw MIRI file downloaded earlier. We downloaded a parameter reference file along with it (**miri_detector1_param_file.asdf**), so you can try either the `run()` or `call()` methods. Note that MIRI data go through some additional steps compared to those we saw for the NIRCam data above. There aren't many parameters beyond those from the NIRCam example to adjust, but feel free to try tweaking some values. Also, try saving the output from the dark current subtraction step.

See [Exercise 1 solutions](#exercise1_solution)

In [ ]:
# Using the run() method:
# Instantiate the pipeline

# Save the final output of the pipeline

# Save the output from the jump step,
# and let's use a more stringent limit for cosmic ray
# detection

# The dark current file used to create the simulated data
# is much different and cleaner than that used in the pipeline,
# so let's turn off the dark subtraction step in order to end up
# with a cleaner rate image.

# Call the run() method


In [ ]:
# Using the call() method:
# First edit the parameter reference file:


In [ ]:
# Use the call() method and supply the parameter reference file


In [ ]:
# Look at one of the output rate files
# HINT: Open one of the rate files and use show_image with
# min and max signal rate limits of 0 and 100.


<a id='detector1_step_by_step'></a>
## Run the individual pipeline steps

In the sections below we run the steps contained within calwebb_detector1 one at a time, in order to more clearly see what each step is doing. Since our example data are from NIRCam, we will skip the MIRI-specific steps.

<a id='dq_init'></a>
### The `Data Quality Initialization` step

#### Summary

This step populates the Data Quality (DQ) mask that is associated with the data file. The DQ flags from the `MASK` reference file are copied into the `PIXELDQ` extension of the input file. A table showing the [mapping of bit values](https://jwst-pipeline.readthedocs.io/en/stable/jwst/references_general/references_general.html#data-quality-flags) in the `MASK` file decribes what types of bad pixels can be flagged. Any other bad pixel types will be ignored.

#### Documentation

[Full description](https://jwst-pipeline.readthedocs.io/en/stable/jwst/dq_init/description.html) of the step.

#### Arguments

There are no optional arguments for this step

#### Reference files used

This step uses the `MASK` reference file. 


#### Run the step

In [ ]:
# Using the run() method. Instantiate and set parameters
dq_init_step = DQInitStep()
dq_init_step.output_dir = output_dir
dq_init_step.save_results = True

# Call the run() method on the uncal file
dq_init = dq_init_step.run(uncal_file)

The step finished without crashing, but as it is said above, there are some warnings worth noting.
Notably, the `NOISY` and `WEIRD` do not correspond to existing `DQ` mnemonics, so they are ignored. This is expected, and means that the `MASK` reference file contains some pixels flagged as `NOISY` and `WEIRD`. Since these bad pixel types are not present in the list of known types of bad pixels, as shown below, these flags are ignored.

In [ ]:
# Print the list of JWST bad pixel types
dqflags.pixel

The pixel values in the `SCI` extension are not changed in this step. Instead, the DQ flags are copied into the `PIXELDQ` extension. The `GROUPDQ` values are not changed in this step. Let's check the `PIXELDQ` values and see what has changed.

We have the datamodel instance of the output available already, but if you wanted to open the output file from the data quality initiailzation step, the output file has the same name as the input file, with the original suffix replaced by "dqinitstep".

In [ ]:
# Generate the DQ step output file from the uncal file name
dq_init_output_file = os.path.join(output_dir, '{}dqinitstep.fits'.format(input_file_base))

In [ ]:
# Print some basic information on the number of flagged pixels
idx_pixelDQ = np.where(dq_init.pixeldq.flatten() == 0.)[0]
num_flagged = dq_init.pixeldq.size - len(idx_pixelDQ)
print('Total pixels in PIXELDQ: {}'.format(dq_init.pixeldq.size))
print('{} pixels have no flags.'.format(len(idx_pixelDQ)))
print('{} pixels ({:.2f}% of the detector) have flags.'.format(num_flagged, num_flagged / dq_init.pixeldq.size))

[Top of Notebook](#top)

<a id='saturation'></a>
## The `Saturation Flagging` step

#### Summary

This step checks the signal values in all pixels across all groups, and adds a [`saturated` flag](https://jwst-pipeline.readthedocs.io/en/stable/jwst/references_general/references_general.html#data-quality-flags) to the `GROUPDQ` extension for pixels and groups where the signal is above the saturation limit.

#### Documentation

[Full description](https://jwst-pipeline.readthedocs.io/en/stable/jwst/saturation/description.html) of the step.

#### Arguments

There are no optional arguments for this step

#### Reference files used

This step uses the [`SATURATION`](https://jwst-pipeline.readthedocs.io/en/stable/jwst/saturation/reference_files.html) reference file. This file contains a map of the saturation threshold in ADU for each pixel on the detector.

#### Run the step

In [ ]:
# Using the run() method
saturation_step = SaturationStep()
saturation_step.output_dir = output_dir
saturation_step.save_results = True

# Call using the the output from the previously-run dq_init step
saturation = saturation_step.run(dq_init)

If there are any saturated values, they should appear in the `GROUPDQ` arrays. Let's examine the `GROUPDQ` data and see if there are any detected:

In [ ]:
# Find indexes of saturated pixels
saturated = np.where(saturation.groupdq & dqflags.pixel['SATURATED'] > 0)

In [ ]:
num_sat_flags = len(saturated[0])
print(('Found {} saturated flags. This may include multiple saturated '
       'groups within a given pixel'.format(num_sat_flags)))

Let's find a pixel that saturated part of the way up the ramp.

In [ ]:
# Create a 4D boolean map of whether the saturation flag is present or not.
saturated = (saturation.groupdq & dqflags.pixel['SATURATED'] > 0)

In [ ]:
# Collapse that down to a 2D map that lists the number of saturated groups 
# for each pixel.
saturated_2d = np.sum(saturated[0, :, :, :], axis=0)

In [ ]:
# Get coordinates of pixels that are saturated in some, but not all, groups.
partial_sat = np.where((saturated_2d > 0) & (saturated_2d < saturated.shape[1]))
print("{} pixels are partially saturated.".format(len(partial_sat[0])))

Now let's choose one of these partially saturated pixels and look at the signal values up the ramp, along with which groups are flagged as saturated.

In [ ]:
sat_y, sat_x = partial_sat
sat_index = 123
y = sat_y[sat_index]
x = sat_x[sat_index]
grps = saturated[0, :, y, x]
print('Saturation flags up the ramp (0 is not saturated, 2 is saturated): {}'
      .format(saturation.groupdq[0, :, y, x]))
print('Pixel signal values up the ramp: {}'.format(saturation.data[0, :, y, x]))

Plot these in order to get a clearer look at the data

In [ ]:
# Get the science and DQ values for the pixel.
groups = np.arange(saturation.data.shape[1])
full_ramp = saturation.data[0, :, y, x]
sat_dq = saturation.groupdq[0, :, y, x].astype(bool)

# Make a copy of the science data and set all saturated groups to NaN
saturated_points = copy.deepcopy(saturation.data[0, :, y, x])
saturated_points[~sat_dq] = np.nan

In [ ]:
# Plot the pixel's values up the ramp and denote the saturated groups
plot_ramps(groups, full_ramp, saturated_points, label1='Not Saturated',
           label2='Saturated', title='Pixel ({}, {})'.format(x, y))

 <a id='superbias'> </a>
## The `Superbias Subtraction` step

#### Summary

This step subtracts the superbias reference frame from each group of the science exposure.

#### Documentation

[Full description](https://jwst-pipeline.readthedocs.io/en/stable/jwst/superbias/description.html) of the step.

#### Arguments

There are no optional arguments for this step

#### Reference files used

This step uses the [`SUPERBIAS`](https://jwst-pipeline.readthedocs.io/en/stable/jwst/superbias/reference_files.html) reference file. This file contains a map of the superbias signal in ADU for each pixel on the detector.


#### Run the step

In [ ]:
# Using the run() method
superbias_step = SuperBiasStep()
superbias_step.output_dir = output_dir
superbias_step.save_results = True

# Call using the the output from the previously-run saturation step
superbias = superbias_step.run(saturation)

Let's visually compare the science products to the raw `uncal` data, looking only at the last group of the first integration.

In [ ]:
# Generate the name of the superbias step's output file from the uncal file name
superbias_output_file = os.path.join(output_dir, '{}superbiasstep.fits'.format(input_file_base))

In [ ]:
# Print the superbias step's output file name
superbias_output_file

In [ ]:
# Look at the shape of the data in the datamodel output from the step
superbias.data[0, 0, :, :].shape

In [ ]:
# Look at the data before and after superbias subtraction
side_by_side(saturation.data[0, 0, :, :], superbias.data[0, 0, :, :], vmin=10000, vmax=18000,
            title1='Before superbias subtraction', title2='After superbias subtraction')

[Top of Notebook](#top)

<a id='refpix'></a>
## The `Reference Pixel Subtraction` step

#### Summary

This step uses the reference pixels, which are not sensitive to illumination, to subtract group- and amplifier-dependent signal originating in the readout electronics from the data. There are two distinct corrections that are applied here.

First, the rows of reference pixels on the top and bottom of the detector are used to subtract amplifier-dependent offsets from each group. Within a given amplifier, the mean value of all reference pixels in even numbered columns is subtracted from the science pixels in the even numbered columns. The same strategy is used for the odd numbered columns. 

The second part of the reference pixel subtraction step uses the reference pixels along the left and right sides of the detector to mitigate 1/f noise. This noise is visible in the data as horizontal banding that stretches across the entire detector. 

#### Documentation

[Full description](https://jwst-pipeline.readthedocs.io/en/stable/jwst/refpix/description.html) of the step.

#### Arguments

[Full details on the optional arguments](https://jwst-pipeline.readthedocs.io/en/stable/jwst/refpix/arguments.html).

#### Reference files used

This step does not use any reference files.


#### Run the step

In order to better show the effects from the 2 parts of the reference file subtraction, we're going to run this step twice. First we'll perform only the mean value subtraction using the top and bottom reference pixels. Then on the second run, we'll perform both the mean value subtraction and the 1/f subtraction.

<a id='refpix_run'></a>
##### Using the run() method

A reminder of the available parameters that can be set:

In [ ]:
# Use the spec attribute to print available parameters
print(RefPixStep.spec)

In [ ]:
# For this 'partial' run, we need to turn off the 1/f correction that
# uses the reference pixels on the sides of the detector. Also, we'll
# save the output using a unique name so as not to confuse the file
# with the output where we run the entire repix subtraction step.

refpix_step_no_sidepix = RefPixStep()
refpix_step_no_sidepix.output_dir = output_dir
refpix_step_no_sidepix.save_results = True
refpix_step_no_sidepix.output_file = 'refpix_test_no_side_pixels'

# Turn off the 1/f correction
refpix_step_no_sidepix.use_side_ref_pixels = False

# Call using the the output from the previously-run superbias step
refpix_no_sidepix = refpix_step_no_sidepix.run(superbias)

Next run the full correction. This will produce the output that we will feed into subsequent steps.

In [ ]:
# Instantiate and set parameters
refpix_step = RefPixStep()
refpix_step.output_dir = output_dir
refpix_step.save_results = True

# Call using the saturation instance from the previously-run
# saturation step
refpix = refpix_step.run(superbias)

<a id='refpix_call'></a>
##### Using the call() method

The following cells show how to run the reference pixel subtraction step with the same parameters as the preceeding two cells, but using the `call()` method rather than the `run()` method. In this case we have a separate parameter reference file for each call. 

Let's look at the parameter referece files. The difference between the two is in the `use_side_ref_pixels` entry, on the bottom line of each. These files look fairly similar to that for the Detector1Pipeline. In this case, the step's parameters and values are all listed in the `parameters` entry of the file.

In [ ]:
# Open the parameter reference file and look at the tree
default_refpix_params = asdf.open(refpix_param_reffile)
default_refpix_params.tree

In [ ]:
# Close the file
default_refpix_params.close()

In [ ]:
# Open the parameter reference file where side reference pixels were not used
# and look at the tree
default_refpix_params = asdf.open(refpix_param_reffile_no_side_pix)
default_refpix_params.tree

In [ ]:
# Close the file
default_refpix_params.close()

Now use the `call()` method and the parameter reference file with all of the default settings to run the refernce pixel subtraction step. If you wish to run these cells, change the cell type to 'Code' in the pull-down menu above, and then execute.

Call the reference pixel subtraction step again, but use the parameter reference file that has the side reference pixel step turned off.

<a id='refpix_strun'></a>
##### From the command line

Here we show the commands that can be used from the command line to run the reference pixel correction step in the same ways as above. In this case, we would have had to save the output from the superbias subtraction step into a fits file, and then provide that file name in the command. For the purposes of this example, assume that the superbias subtraction output file is called superbias_sub.fits.

<div class="alert alert-block alert-info">
    
```
strun refpix_paramfile.asdf superbias_sub.fits
````
    
```
strun refpix_no_side_pix_paramfile.asdf superbias_sub.fits
```
    
</div>

#### Examine the output

If you wish to examine the output file from this step, the file is saved with the "refpixstep" suffix.

In [ ]:
# Define the reference pixel step output filename
refpix_output_file = os.path.join(output_dir, '{}refpixstep.fits'.format(input_file_base))

Let's explore the output from this step. As with all NIR detectors, the outermost 4 rows and columns comprise the reference pixels.

Let's use the datamodels from before and after the reference pixel subtraction to have a look at the data. 

We'll zoom in on the top few rows in order to see the changes. Note how the "odd/even" effect dominates the signal prior to reference pixel subtraction. Even numbered columns and odd numbered columns have significantly different signal levels.

In [ ]:
# Side by side, before/after reference pixel subtraction
side_by_side(superbias.data[0, 5, 2030:, 1030:1050], refpix.data[0, 5, 2030:, 1030:1050],
             vmin=0, vmax=30000, title1='Before Refpix Subtraction',
             title2='After Refpix Subtraction')

In [ ]:
# Image of the difference before/after reference pixel subtraction
show_image(superbias.data[0, 5, 2030:, 1030:1050] - refpix.data[0, 5, 2030:, 1030:1050],
           vmin=7000, vmax=16000, title='Difference: Before - After Refpix Subtraction')

Now let's look at the difference in the data after the mean value subtraction compared to the case where both the mean value subtraction and the 1/f correction are done.

In [ ]:
# Side by side view of the reference pixel subtraction with and without
# using the side reference pixels to subtract 1/f noise
side_by_side(refpix_no_sidepix.data[0, 5, :, :], refpix.data[0, 5, :, :],
             vmin=20, vmax=150, title1='No Side Repix Correction',
             title2='With Side Refpix Correction')

In [ ]:
# Look at the difference image before and after the side reference
# pixels are used to subtract the 1/f noise
show_image(refpix.data[0, 5, :, :] - refpix_no_sidepix.data[0, 5, :, :],
           vmin=-10, vmax=10, title="Difference with/without using side refpix")

[Top of Notebook](#top)

<a id='exercise2'></a>
### Exercise 2

Try running the reference pixel subtraction step yourself! Try adjusting the number of rows of reference pixels that are smoothed as the step attempts to subtract 1/f noise. 

**HINT:** remember to use `RefPixStep.spec` to see the names of available parameters as well as their default values. Since we didn't explicitly set the smoothing length in our calls above, the default value was used. You can run the step using the `superbias` data model object, as we did above, or you can run on the `superbias_output_file` that we saved when we ran the SuperBiasStep.

See [Exercise 2 solution](#exercise2_solution)

In [ ]:
# Check the names of the available parameters


In [ ]:
# Using the run() method.

# Instantiate the step


# Set the side_smoothing_length to a non-default value
# (default is 11)


# Call using the superbias instance from the previously-run
# saturation step


In [ ]:
# Using the call() method

# First open 'refpix_paramfile.asdf' and set the 
# side_smoothing_length entry to a new value.


<a id='linearity'></a>
## The `Linearity Correction` step 

#### Summary

This step applies the classical linearity correction to the data on a pixel-by-pixel, integration-by-integration, group-by-group manner.

#### Documentation

[Full description](https://jwst-pipeline.readthedocs.io/en/stable/jwst/linearity/description.html) of the step.

#### Arguments

There are no optional arguments for this step

#### Reference files used

This step uses the [`LINEARITY`](https://jwst-pipeline.readthedocs.io/en/stable/jwst/linearity/reference_files.html) reference file. This file contains the polynomial coefficients used to apply the linearity correction to non-linear data.


#### Run the step

In [ ]:
# Using the run() method
linearity_step = LinearityStep()
linearity_step.output_dir = output_dir
linearity_step.save_results = True

# Call using the refpix instance from the previously-run
# refpix step
linearity = linearity_step.run(refpix)

#### Examine the output

The output file from this step has the "linearitystep" suffix.

In [ ]:
linearity_output_file = os.path.join(output_dir, '{}linearitystep.fits'.format(input_file_base))

Let's look at the signal up the ramp for a high signal pixel, in order to more easily see how the linearity correction changed the data.

In [ ]:
# Using the 3rd group, find the difference between the data before and after
# linearity correction. Find the pixels where this difference is greater than
# 20 DN, and also where the signal in the final group is over 40,000 DN.
lin_fix = linearity.data[0, 3, :, :] - refpix.data[0, 3, :, :]
well_exposed = np.where((linearity.data[0, -1, :, :] > 40000.) & (lin_fix > 20))

In [ ]:
print('{} pixels meet the criteria above.'.format(len(well_exposed[0])))

Pick one of these pixels and plot the signal before and after the linearity correction.

In [ ]:
index = 3
lin_pix_x, lin_pix_y = (well_exposed[1][index], well_exposed[0][index])

In [ ]:
# Create an array of group numbers to plot against
group_nums = np.arange(10)

In [ ]:
plot_ramps(group_nums, refpix.data[0, :, lin_pix_y, lin_pix_x],
           linearity.data[0, :, lin_pix_y, lin_pix_x], label1='Uncorrected', label2='Corrected',
           title='Pixel ({}, {})'.format(lin_pix_x, lin_pix_y))

In this case, the pixel reached saturation in group 9. So in group 9, the linearity correction made no changes. Between groups 0 to 8, you can see the original signal (in black) becoming more and more non-linear as signal increases, along with how the linearity correction modified the signal (red).

[Top of Notebook](#top)

<a id='persistence'></a>
## The `Persistence Correction` step 

#### Summary

This step uses a model to calculate the amount of signal in each group of each pixel that comes from persistence. This persistence signal is then subtracted, pixel-by-pixel and group-by-group, from the data.

#### Documentation

[Full description](https://jwst-pipeline.readthedocs.io/en/stable/jwst/persistence/description.html) of the step.

#### Arguments

[Optional arguments](https://jwst-pipeline.readthedocs.io/en/stable/jwst/persistence/arguments.html) for this step include setting the threshold signal value above which pixels are flagged in the DQ extension, as well as saving the subtracted persistence signal in a separate file.

#### Reference files used

This step uses the [`TRAPDENSITY`, `PERSAT`, and `TRAPPARS`](https://jwst-pipeline.readthedocs.io/en/stable/jwst/persistence/reference_files.html) reference files. The TRAPDENSITY file contains a map of the relative number of traps per pixel. The PERSAT reference file contains a map of the persistence saturation level, and the TRAPPARS reference file contains parameters related to the persistence calculation model. 

#### Run the step

List the available parameters:

In [ ]:
print(PersistenceStep.spec)

In [ ]:
# Using the run() method
persist_step = PersistenceStep()
persist_step.output_dir = output_dir
persist_step.save_results = True

# Specify the trapsfilled file, which contains the
# state of the charge traps in the preceding exposure
persist_step.input_trapsfilled = persist_file

# Let's also save a separate file that contains the
# subtracted persistence signal 
persist_step.save_persistence = True

# Call using the refpix instance from the previously-run
# linearity step
persist = persist_step.run(linearity)

#### Examine the output

The primary output file from this step has the "persistencestep.fits" suffix added.

In [ ]:
# Name of the file containing persistence-subtracted data
persist_output_file = os.path.join(output_dir, '{}persistencestep.fits'.format(input_file_base))

Let's look at the optional output file that contains the subtracted persistence signal. 

<div class="alert alert-block alert-warning">NOTE: In this case the output is pretty boring. It turns out that the trap density map reference file for NIRCam is empty because it is not yet well characterized. This means that the persistence signal is zero in all pixels for all exposures. Once the map is updated in commissioning, this step will start calculating persistence values.</dev>

In [ ]:
# Name of the file containing a map of the calculated persistence signal
persist_signal_file = os.path.join(output_dir, '{}output_pers.fits'.format(input_file_base))

In [ ]:
# Read in the map of persistence signal
persist_signal = fits.getdata(persist_signal_file)

The file contains the persistence signal for each group of each integration, as we can see by the shape.

In [ ]:
persist_signal.shape

In [ ]:
np.min(persist_signal), np.max(persist_signal)

Let's look at the persistence signal in the final group.

In [ ]:
show_image(persist_signal[0, -1, :, :], 0, .01)

<a id='dc'></a>
##  The `Dark Current Subtraction` step 

#### Summary

This step subtracts the dark current, group by group, from the input integrations.

#### Documentation

[Full description](https://jwst-pipeline.readthedocs.io/en/stable/jwst/dark_current/description.html) of the step.

#### Arguments

There are no optional arguments for this step

#### Reference files used

This step uses the [`DARK`](https://jwst-pipeline.readthedocs.io/en/stable/jwst/dark_current/reference_files.html) reference file. This file contains the measured mean dark current associated with the detector and subarray.


#### Run the step

NOTE: This is one of the longer-running steps of calwebb_detector1. 

In [ ]:
# Using the run() method
dark_step = DarkCurrentStep()
dark_step.output_dir = output_dir
dark_step.save_results = True

# Call using the persistence instance from the previously-run
# persistence step
dark = dark_step.run(persist)

In [ ]:
dark_output_file = os.path.join(output_dir, '{}darkcurrentstep.fits'.format(input_file_base))

[Top of Notebook](#top)

<a id='jump'></a>
## The `Cosmic Ray Flagging` step

#### Summary

This step searches for "jumps" in the ramp data. In this case, a jump in a pixel's ramp is defined as a large deviation in the count rate relative to that in the other groups. When a jump is found, the associated flag is added to the `GROUPDQ` extension for the group and pixel where the jump was detected. The science data are not modified at all. In the subsequent ramp-fitting step, the algorithm will look into the `GROUPDQ` array and ignore any groups where the jump flag has been set.

#### Documentation

[Full description](https://jwst-pipeline.readthedocs.io/en/stable/jwst/jump/description.html) of the step.

#### Arguments

The jump step has [several optional arguments](https://jwst-pipeline.readthedocs.io/en/stable/jwst/jump/arguments.html)

#### Reference files used

This step uses the [`READNOISE`](https://jwst-pipeline.readthedocs.io/en/stable/jwst/references_general/readnoise_reffile.html) and [`GAIN`](https://jwst-pipeline.readthedocs.io/en/stable/jwst/references_general/gain_reffile.html) reference files. These files contain maps of the readnoise and gain values across the detector.


#### Run the step

List the available parameters:

In [ ]:
print(JumpStep.spec)

In [ ]:
# Using the run() method
jump_step = JumpStep()
jump_step.output_dir = output_dir
jump_step.save_results = True
jump_step.rejection_threshold = 9

# Call using the dark instance from the previously-run
# dark current subtraction step
jump = jump_step.run(dark)

#### Examine the output

In [ ]:
jump_output_file = os.path.join(output_dir, '{}jumpstep.fits'.format(input_file_base))

Let's see what some of these jumps look like:

In [ ]:
# How many total jump flags were added? Note that some pixels
# will have more than one group flagged with a jump.
jump_flags = np.where(jump.groupdq & dqflags.pixel['JUMP_DET'] > 0)
print('{} jump flags detected.'.format(len(jump_flags[0])))

# Create a 4-dimensional map of the jump flags
jump_map = (jump.groupdq & dqflags.pixel['JUMP_DET'] > 0)

# Collapse down to a 2D map of the number of flagged jumps in each pixel
jump_map_2d = np.sum(jump_map[0, :, :, :], axis=0)

# Determine how many pixels have jump flags
jump_map_indexes = np.where(jump_map_2d > 0)
impacted_pix = np.sum(jump_map_2d > 0)
total_pix = 2048 * 2048
print(('{} pixels ({:.2f}% of the detector) have been flagged with '
      'at least one jump.'.format(impacted_pix, 100. * impacted_pix / total_pix)))

Plot some of the pixels impacted by jumps. The red marks signify flagged jumps. These signal values will be ignored in subsequent ramp-fitting.

In [ ]:
# The jump map is 4-dimensional, just like the science data
jump_map.shape

In [ ]:
# Create an array of group numbers to plot against
group_indexes = np.arange(jump_map.shape[1]).astype(int)

In [ ]:
# Pick one pixel with a flagged jump, and find the group(s) with the jump flags
j_index = 10112
jumpy = jump_map_indexes[0][j_index]
jumpx = jump_map_indexes[1][j_index]
jump_grp = jump_map[0, :, jumpy, jumpx]
print('Jump located in group(s) {} of pixel ({}, {})'.format(group_indexes[jump_grp], jumpx, jumpy))

In [ ]:
# Plot the signal up the ramp for this pixel
plot_jump(jump.data[0, :, jumpy, jumpx], jump_grp, xpixel=jumpx, ypixel=jumpy)

Plot a grid of some examples of flagged jumps. The red marks signify flagged jumps. These signal values will be ignored in subsequent ramp-fitting.

In [ ]:
indexes_to_plot = [200, 401, 600, 30010, 31000, 1202, 1400, 21600, 10112]
jump_data = np.zeros((jump.shape[1], len(indexes_to_plot)))
jump_grps = np.zeros((jump.shape[1], len(indexes_to_plot))).astype(bool)
jump_locs = []
for counter, idx in enumerate(indexes_to_plot):
    #integ, grp, y, x = jump_flags[idx]
    y = jump_map_indexes[0][idx]
    x = jump_map_indexes[1][idx]
    grp = jump_map[0, :, y, x]

    jump_data[:, counter] = jump.data[0, :, y, x]
    jump_grps[:, counter] = grp
    jump_locs.append((x, y))

In [ ]:
plot_jumps(jump_data, jump_grps, jump_locs)

[Top of Notebook](#top)

<a id='ramp_fitting'></a>
## The `Ramp Fitting` step

#### Summary

This step performs line-fitting to the corrected data, and produces a slope image for each integration. For a given pixel, any groups within an integration that contain a jump flag or that are flagged as saturated are ignored.

For the purposes of this notebook, we will use the `save_opt` parameter to tell the ramp-fitting step to save an optional output file that contains some of the details on the ramp fits. This information will be used for the plots after the step is run. By default, `save_opt` is False and the optional outputs are not saved.


#### Documentation

[Full description](https://jwst-pipeline.readthedocs.io/en/stable/jwst/ramp_fitting/description.html) of the step.

#### Arguments

The jump step has [several optional arguments](https://jwst-pipeline.readthedocs.io/en/stable/jwst/ramp_fitting/arguments.html), including the ability to save optional outputs into a second file.

#### Reference files used

This step uses the [`READNOISE`](https://jwst-pipeline.readthedocs.io/en/stable/jwst/references_general/readnoise_reffile.html) and [`GAIN`](https://jwst-pipeline.readthedocs.io/en/stable/jwst/references_general/gain_reffile.html) reference files. These files contain maps of the readnoise and gain values across the detector.


#### Run the step

List the available parameters:

In [ ]:
print(RampFitStep.spec)

In [ ]:
# Using the run() method
ramp_fit_step = RampFitStep()
ramp_fit_step.output_dir = output_dir
ramp_fit_step.save_results = True

# Let's save the optional outputs, in order
# to help with visualization later
ramp_fit_step.save_opt = True

# Call using the dark instance from the previously-run
# jump step
ramp_fit = ramp_fit_step.run(jump)

An important detail here is that there are two output files: 

1. The file ending with `*_0_rampfitstep.fits` contains the mean slope image across all integrations in the exposure.

2. The file ending with `*_1_rampfitstep.fits` contains a separate slope image for each integration. In this case our exposure contains only a single integration, so the data in the two files are identical.

In [ ]:
rampfit_output_file = os.path.join(output_dir, '{}_0_rampfitstep.fits'.format(input_file_base))

We are working with the output datamodels in this notebook. Unlike the preceeding steps, where the output was a single datamodel, the ramp_fitting step outputs a tuple of 2 datamodel instances. The first element in the tuple is the datamodel instance containing the mean rate image, while the second element is the datamodel instance containing a separate slope image for each integration.

In [ ]:
type(ramp_fit)

In [ ]:
type(ramp_fit[0]), type(ramp_fit[1])

Show the shape of the two output data models. The output with the mean slope image is only 2-dimensional, while the output with one slope image for each integration is 3-dimensional, even in this case where we have only one integration.

In [ ]:
ramp_fit[0].shape, ramp_fit[1].shape

Here we see that in our case with a single integration, the two datamodel instances contain identical data.

In [ ]:
ramp_fit[0].data[500, 500], ramp_fit[1].data[0, 500, 500]

And note that the data in these files are the slopes from the ramp fitting, so the units have changed from DN to DN/sec.

In [ ]:
# Show the units of the ramp-fit data
ramp_fit[0].meta.bunit_data

Let's load the data from the third output file, which contains the optional outputs. Our goal is to grab the intercept values from the line-fitting and use them to re-create the plots from the jump step and overplot the best-fit signals on top of the signals that went into the step.

In [ ]:
# Generate the name of the optional output file
optional_file = os.path.join(output_dir, '{}fitopt.fits'.format(input_file_base))

In [ ]:
# Open the file and examine the extensions
hdulist = fits.open(optional_file)
hdulist.info()

The intercepts from the line-fitting are stored in the `YINT` extension.

In [ ]:
hdulist['YINT'].data[0, :, 200, 200]

For our plots below, we need the intercept values from the line-fitting. Let's ignore all but the first plane of the extension, since those are zeros.

In [ ]:
intercepts = hdulist['YINT'].data[0, 0, :, :]
hdulist.close()

For the pixels to be plotted, create linear ramps using the output slopes and intercepts

In [ ]:
# Get the exposure time associated with each group
num_groups = ramp_fit[0].meta.exposure.ngroups
group_time = ramp_fit[0].meta.exposure.group_time
group_times = np.arange(num_groups) * group_time

In [ ]:
# Reconstruct linear ramps from the slope and intercept values
lin_ramps = np.zeros((jump.shape[1], len(indexes_to_plot)))
for counter, idx in enumerate(indexes_to_plot):
    y = jump_map_indexes[0][idx]
    x = jump_map_indexes[1][idx]
    grp = jump_map[0, :, y, x]

    rate = ramp_fit[0].data[y, x]
    intercept = intercepts[y, x]
    lin_ramps[:, counter] = intercept + (rate * group_times)    

In [ ]:
# Reconstruct the linear ramp for the single pixel we showed after the jump step
lin_data = intercepts[jumpy, jumpx] + (ramp_fit[0].data[jumpy, jumpx] * group_times)

In [ ]:
# Plot again the single pixel from the jump step, along with its
# reconstructed best-fit linear fit
plot_jump(jump.data[0, :, jumpy, jumpx], jump_grp, xpixel=jumpx,
          ypixel=jumpy, slope=lin_data)

In [ ]:
# Plot the same as above, but for the collection of pixels
# we plotted after the jump step
plot_jumps(jump_data, jump_grps, jump_locs, slopes=lin_ramps)

Look at the slope image. 

In [ ]:
show_image(ramp_fit[0].data, 0, 10)

What's going on with the dark pixels scattered across the left side of the detector? It turns out they have signal values of exactly zero in the slope image. Let's have a closer look, and ignore the reference pixels.

In [ ]:
scipix = ramp_fit[0].data[4:2044, 4:2044]
zero_pix = np.where(scipix == 0.0)
print('{} science pixels have a slope that is exactly zero.'.format(len(zero_pix[0])))

Pick one of these pixels, and examine the pixel's signals up the ramp, as well as its data quality flags up the ramp.

In [ ]:
idx = 45

# Add 4 to the coordinates because we stripped off the 4 columns/rows of 
# refpix in the np.where statement above
y = 4 + zero_pix[0][idx]
x = 4 + zero_pix[1][idx]
print('({}, {}) has a slope of 0.'.format(x, y))

Print the slope (should be zero), signal values up the ramp, and DQ flags up the ramp

In [ ]:
scipix[y-4, x-4]

In [ ]:
linearity.data[0, :, y, x]

In [ ]:
linearity.groupdq[0, :, y, x]

What does a DQ flag value of 2 mean?

In [ ]:
# Print JWST bad pixel flag definitions
dqflags.pixel

So this pixel (and the others with values of 0 in the slope image) have been flagged as saturated in all groups. In this case, the pipeline cannot calculate a slope value, and assigns a value of zero. We'll see in the Stage 3 notebook how these pixels will be ignored when combining multiple exposures to create a final mosaic image.

[Top of Notebook](#top)

## Bonus Topic: Logging

We've seen in the calls to the pipeline and steps above that the collection of logging entries output to the screen can be quite long. If you wish to capture this information in a file rather than on the screen, then you can create a simple logging configuration file that resides in the directory where you are calling the pipeline. By setting the `logcfg` keyword to the name of this file in your pipeline calls, the logging entries will be routed to the file specified in the configuration file.

See the [log file set up instructions](https://jwst-pipeline.readthedocs.io/en/latest/jwst/introduction.html#logging-configuration) for details.

<a id='exercise_solutions'></a>
## Exercise Solutions

<a id='exercise1_solution'></a>
### Exercise 1: Run MIRI data through the pipeline

Using the supplied MIRI exposure (which can be downloaded in the [Download Data](#download_data) section), run the calwebb_detector1 pipeline, examnine which steps are run, and note the different order compared to the pipeline run above.

In [ ]:
# Using the run() method:
# Instantiate the pipeline
miri1 = calwebb_detector1.Detector1Pipeline()

# Save the final output of the pipeline
miri1.output_dir = output_dir
miri1.save_results = True

# Save the output from the jump detecion step,
# and let's use a more stringent limit for cosmic ray
# detection
miri1.jump.save_results = True
miri1.jump.rejection_threshold = 3

# The dark current file used to create the simulated data
# is much different and cleaner than that used in the pipeline,
# so let's turn off the dark subtraction step in order to end up
# with a cleaner rate image.
miri1.dark_current.skip = True

# Call the run() method
miri_output = miri1.run(miri_uncal_file)

In [ ]:
# Using the call() method:
# In order to match the parameters used in the the call to run() above,
# you first must edit the parameter reference file, which is:
miri_det1_param_reffile

Add the following lines to the bottom of the file:

```
- class: jwst.dark_current.dark_current_step.DarkCurrentStep
  name: dark_current
  parameters: {skip: True}
- class: jwst.jump.jump_step.JumpStep
  name: jump
  parameters: {rejection_threshold: 3,
               save_results: True}
```

In [ ]:
# Now use the call() method and supply the parameter reference file
miri_output = calwebb_detector1.Detector1Pipeline.call(miri_uncal_file,
                                                       output_dir=output_dir,
                                                       save_results=True,
                                                       config_file=miri_det1_param_reffile)

In [ ]:
# Look at one of the output rate files
# HINT: Open one of the rate files and use show_image with
# min and max signal rate limits of 0 and 100.
miri_rate_file = miri_uncal_file.replace('uncal.fits', 'rate.fits')
miri_rate = datamodels.open(miri_rate_file)

In [ ]:
show_image(miri_rate.data, 0, 100)

Back to [Exercise 1](#exercise1)

<a id='exercise2_solution'></a>
### Exercise 2: Re-run the `reference pixel subtraction` step

Try re-running the [reference pixel subtraction](#refpix) step and changing the `side_smoothing_length` and/or `side_gain` parameters. Examine the output to see how well the 1/f noise is removed.

In [ ]:
# Check the names of the available parameters
RefPixStep.spec

In [ ]:
# Using the run() method.

# Instantiate the step
refpix_step = RefPixStep()

# Set the side_smoothing_length to a non-default value
# (default is 11)
refpix_step.side_smoothing_length = 51

# Call using the superbias instance from the previously-run
# saturation step
refpix_ex2 = refpix_step.run(superbias)
# OR run using the output file from the superbias step
# refpix_ex2 = refpix_step.run(superbias_output_file)

In [ ]:
# Using the call() method

# First open 'refpix_paramfile.asdf' and set the 
# side_smoothing_length entry to a new value.
refpix_ex2 = RefPixStep.call(superbias, output_dir=output_dir,
                                save_results=True, 
                                config_file=refpix_param_reffile)

# OR run using the output file from the superbias step
#refpix_ex2 = RefPixStep.call(superbias_output_File, output_dir=output_dir,
#                                save_results=True, 
#                                config_file=refpix_param_reffile)

Back to [Exercise 2](#exercise2)

That is the end of the Stage 1 pipeline. We have now transformed a single exposure from a raw, multiaccum ramp into a signal rate image after applying detector-level corrections. From here, the data move on to the Stage 2 pipeline. For imaging data such as these, that means the *calwebb_image2* pipeline. This will be shown in the Stage 2 notebook. 

[Top of Notebook](#top)